### POC of Selenium to automate Adding Certificates

Based on the example used in [this article](https://www.browserstack.com/guide/python-selenium-to-run-web-automation-test), I've included the sample and then expanded to read and use data from `LMChain.pem` which holds 43 certificates used as CA Roots by LM.

A scond article used in building the test case [helped with submitting the form](https://www.tutorialspoint.com/what-are-the-ways-of-submitting-a-form-in-selenium-with-python).

I chose to the `Chrome` option as I had issues invoking the `Firefox` driver on my laptop.  I may come back to that, but in the end, it doesn't really matter for the work we are doing.

----------


### On to Automation!

Now that we've walked through the example code, it's time to make it interesting by introducing `LMChain.pem` and figuing out how to use it to drive a process to add all the contained certs to the ExtraHop `Trusted Certificates` pool.

-----

I decided to use `split('-----END CERTIFICATE-----')` to put each certificatae into a `LIST` element that we could iterate through.  I originally through I'd need to get the Begin...End pair down to a single character to get the split to work.  No need to go back and figure out the Begin line, as we need that anyway when we pass the cert into ExtraHop for processing.

In [25]:
import pathlib
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [1]:
with open("C:/Users/e309516/LMChain.pem") as f:
    content = f.read()

#print(content[1:40])


In [2]:
#fix=content.replace('-----BEGIN CERTIFICATE-----','')
#fix=fix.replace('-----END CERTIFICATE-----','|')
#certs = fix.split('|')

certs = content.split('-----END CERTIFICATE-----')

In [8]:
### No real need for this ongoing, it's just a bit of proof and verification that we have the certs.
print(len(certs))
for i in range(0, len(certs)-1):
    print(i, certs[i][28:40].strip())

44
0 MIIFXTCCA8Wg
1 MIIFXzCCA8e
2 MIIEXjCCA0a
3 MIIFYTCCA8m
4 MIIFVjCCA76
5 MIIFijCCBHK
6 MIIFojCCBAq
7 MIIFojCCBAq
8 MIIFZDCCA8y
9 MIIIijCCB3K
10 MIIFYzCCA8u
11 MIIIczCCB1u
12 MIIESDCCAzC
13 MIIFSDCCA7C
14 MIIEZzCCA0+
15 MIIFVjCCA76
16 MIIFVjCCA76
17 MIIFVjCCA76
18 MIIFVjCCA76
19 MIIFVjCCA76
20 MIIFVjCCA76
21 MIIFVjCCA76
22 MIIFVzCCA7+
23 MIIFVjCCA76
24 MIIFVzCCA7+
25 MIIFVzCCA7+
26 MIIFVzCCA7+
27 MIIFVzCCA7+
28 MIIFVjCCA76
29 MIIFVjCCA76
30 MIIFVjCCA76
31 MIIFVjCCA76
32 MIIFVjCCA76
33 MIIFVjCCA76
34 MIIFVjCCA76
35 MIIFVzCCA7+
36 MIIFVzCCA7+
37 MIIFVjCCA76
38 MIIFVjCCA76
39 MIIFVzCCA7+
40 MIIFVzCCA7+
41 MIIFVzCCA7+
42 MIIIqDCCB5C


------

### We have certs, lets get the web page ready


--------------

In [29]:
b = webdriver.Chrome()
b.get('https://eda-cloud-comm-au-southeast-2.intl.lmco.com/admin/trustedcertificates')
print(b.title)

ExtraHop


-----
### Loop here

We could loop through the certificates in the LMChain.pem file here.

this POC just clicks the `Add New` button and then pushes a new entry with the current cert.  Using a lame naming scheme of **LMCACert<num>**.  seems the easiest.  I imagine you could decode the cert and pull out the name and use that, but for now I would stick with the basics.

The input fields in the New Cert Form are `name` and `cert`.  We populate those and then `Submit` the form.

I would insert a `sleep` just to keep it from possibly overwhelming the device and be more akin to a person entering the data.

--------

Steps:
 - first step is to "Click" the `New Cert` button.
 - on New cert page - load name and cert fields
 - submit the form
 - repleat



In [31]:

for i in range(0, len(certs)-1):
    time.sleep(1)
    b.find_element_by_class_name('buttonlink').click()               # Choose "Add Certificate"
    b.find_element_by_name('name').send_keys('LMCA'+str(i+1))
    b.find_element_by_name('cert').send_keys(str(certs[i])+'-----END CERTIFICATE-----\n')
    time.sleep(1)
    b.find_element_by_xpath('/html/body/main/div/div/form').submit()  # submit new certificate

In [24]:
b.close()